In [1]:
dataset_name = "W19_comb"
df_list = [ "BES_Panel", "BES_reduced_with_na"]

In [2]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

%matplotlib inline

import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import pickle, os, gc, re

sns.set();
sns.set_palette("colorblind")

from IPython.display import display, display_html, HTML
from IPython.core.debugger import set_trace
# plt.rcParams["axes.grid"] = False

import Jupyter_module_loader
from utility import *
import gaussian_kde

import warnings
warnings.filterwarnings('ignore')

import holoviews as hv
from holoviews import opts

encoding = "ISO-8859-1"

# you should clone this git to a subdirectory called 'BES_analysis_code' (in some directory - I call it BES_analysis - doesn't matter though)
# %matplotlib inline
(BES_code_folder, BES_small_data_files, BES_data_folder,
 BES_output_folder, BES_file_manifest, BES_R_data_files) = setup_directories()

global BES_Panel, BES_numeric, BES_reduced, BES_reduced_with_na, BES_non_numeric
data_subfolder = BES_data_folder + dataset_name + os.sep

(manifest, dataset_filename, dataset_description, dataset_citation,
 dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)

for df in df_list:
    if df=="BES_Panel":
        globals()[df]  = pd.read_pickle(data_subfolder + dataset_filename.replace('.dta','.zip'),compression='zip')
    else:
        globals()[df]  = pd.read_pickle(data_subfolder + df + '.zip',compression='zip' )
        globals()[df].replace(-1,np.nan,inplace=True)
  
(var_type, cat_dictionary, new_old_col_names, old_new_col_names) = get_small_files(data_subfolder, encoding)

# get full set of inferred "cross wave" auth-lib/left-right values and ages
pan_dataset_allr_values = pd.read_csv(BES_small_data_files + "pan_dataset_allr_values"+".csv")
pan_dataset_ages = pd.read_csv( BES_small_data_files + "pan_dataset_ages"+".csv" )

Toggle code

var_type (7911, 14)


In [76]:
immigSelfW8  = search(BES_reduced_with_na,"immigSelfW8" ).index[0]
immigSelfW9  = search(BES_reduced_with_na,"immigSelfW9" ).index[0]
immigSelfW10 = search(BES_reduced_with_na,"immigSelfW10").index[0]
D8_10immigSelf = BES_reduced_with_na[immigSelfW10]-BES_reduced_with_na[immigSelfW8]
D9_10immigSelf = BES_reduced_with_na[immigSelfW10]-BES_reduced_with_na[immigSelfW9]
D8_9immigSelf = BES_reduced_with_na[immigSelfW9]-BES_reduced_with_na[immigSelfW8]


changeImmigW8  = search(BES_reduced_with_na,"changeImmigW8" ).index[0]
changeImmigW10 = search(BES_reduced_with_na,"changeImmigW10").index[0]
D8_10changeImmig = BES_reduced_with_na[changeImmigW10]-BES_reduced_with_na[changeImmigW8]

changeCrimeW8  = search(BES_reduced_with_na,"changeCrimeW8" ).index[0]
changeCrimeW10 = search(BES_reduced_with_na,"changeCrimeW10").index[0]
D8_10changeCrime = BES_reduced_with_na[changeCrimeW10]-BES_reduced_with_na[changeCrimeW8]

immigEconW8  = search(BES_reduced_with_na,"immigEconW8" ).index[0]
immigEconW10 = search(BES_reduced_with_na,"immigEconW10").index[0]
D8_10immigEcon = BES_reduced_with_na[immigEconW10]-BES_reduced_with_na[immigEconW8]

immigCulturalW8  = search(BES_reduced_with_na,"immigCulturalW8" ).index[0]
immigCulturalW10 = search(BES_reduced_with_na,"immigCulturalW10").index[0]
D8_10immigCultural = BES_reduced_with_na[immigCulturalW10]-BES_reduced_with_na[immigCulturalW8]

controlImmigW8  = search(BES_reduced_with_na,"controlImmigW8" ).index[0]
controlImmigW9  = search(BES_reduced_with_na,"controlImmigW9" ).index[0]
controlImmigW10 = search(BES_reduced_with_na,"controlImmigW10").index[0]
D9_10controlImmig = BES_reduced_with_na[controlImmigW10]-BES_reduced_with_na[controlImmigW9]

# df = pd.concat([D8_10immigSelf,D8_10changeImmig,D8_10changeCrime,D8_10immigEcon,D8_10immigCultural],axis=1).dropna()
# df.columns = ["D8_10immigSelf","D8_10changeImmig","D8_10changeCrime","D8_10immigEcon","D8_10immigCultural"]

In [77]:
from linearmodels.iv import IV2SLS
from linearmodels.iv import compare
import statsmodels.api as sm
from statsmodels.api import add_constant

In [78]:
df.shape

(12273, 25)

In [85]:
df = pd.concat([D8_10immigSelf,D8_9immigSelf,D9_10immigSelf,D8_10changeImmig,D8_10changeCrime,D8_10immigEcon,D8_10immigCultural,D9_10controlImmig],axis=1).dropna().astype('float32')
df.columns = ["D8_10immigSelf","D8_9immigSelf","D9_10immigSelf","D8_10changeImmig","D8_10changeCrime","D8_10immigEcon","D8_10immigCultural","D9_10controlImmig"]
df = add_constant(df, has_constant='add')

df["immigSelfW8"] = BES_reduced_with_na[immigSelfW8]
df["immigSelfW9"] = BES_reduced_with_na[immigSelfW9]
df["immigSelfW10"] = BES_reduced_with_na[immigSelfW10]

df["changeImmigW8"] = BES_reduced_with_na[changeImmigW8]
df["changeImmigW10"] = BES_reduced_with_na[changeImmigW10]

df["changeCrimeW8"] = BES_reduced_with_na[changeCrimeW8]
df["changeCrimeW10"] = BES_reduced_with_na[changeCrimeW10]

df["immigEconW8"] = BES_reduced_with_na[immigEconW8]
df["immigEconW10"] = BES_reduced_with_na[immigEconW10]

df["immigCulturalW8"] = BES_reduced_with_na[immigCulturalW8]
df["immigCulturalW10"] = BES_reduced_with_na[immigCulturalW10]

df["controlImmigW8"] = BES_reduced_with_na[controlImmigW8]
df["controlImmigW9"] = BES_reduced_with_na[controlImmigW9]
df["controlImmigW10"] = BES_reduced_with_na[controlImmigW10]


df["changeImmigW8_cat"] = BES_Panel["changeImmigW8"]
df["changeImmigW8_cat"] = df["changeImmigW8_cat"].cat.reorder_categories( list(BES_Panel["changeImmigW8"].value_counts().index) )

df["changeImmigW10_cat"] = BES_Panel["changeImmigW10"]
df["changeImmigW10_cat"] = df["changeImmigW10_cat"].cat.reorder_categories( list(BES_Panel["changeImmigW10"].value_counts().index) )


df["controlImmigW8_cat"] = BES_Panel["controlImmigW8"]
df["controlImmigW8_cat"] = df["controlImmigW8_cat"].cat.reorder_categories( list(BES_Panel["controlImmigW8"].value_counts().index) )

df["controlImmigW9_cat"] = BES_Panel["controlImmigW9"]
df["controlImmigW9_cat"] = df["controlImmigW9_cat"].cat.reorder_categories( list(BES_Panel["controlImmigW9"].value_counts().index) )

df["controlImmigW10_cat"] = BES_Panel["controlImmigW10"]
df["controlImmigW10_cat"] = df["controlImmigW10_cat"].cat.reorder_categories( list(BES_Panel["controlImmigW10"].value_counts().index) )




In [38]:
# 2 stage least squares resolves to OLS when Endongeous and Instrument arguments are set to None
dep = "D8_10immigSelf"
exog = ['const','D8_10changeImmig']

res_ols = IV2SLS(dependent = df[dep],
                 exog = df[exog],
                 endog = None,
                 instruments = None).fit(cov_type='unadjusted')
print(res_ols)

# there's an effect of DchangeImmig in DimmigSelf ... but it's quite a small contribution

                            OLS Estimation Summary                            
Dep. Variable:         D8_10immigSelf   R-squared:                      0.0065
Estimator:                        OLS   Adj. R-squared:                 0.0064
No. Observations:               14901   F-statistic:                    97.465
Date:                Wed, Apr 15 2020   P-value (F-stat)                0.0000
Time:                        19:40:33   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                                Parameter Estimates                                 
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
const                0.0391     0.0167     2.3442     0.0191      0.0064      0.0719
D8_10changeImmig    -0.1551 

In [39]:
# First stage to check whether DchangeCrime is a strong instrument for changeImmig

dep = "D8_10changeImmig"
exog = ['const','D8_10changeCrime']

res_first = IV2SLS(dependent = df[dep],
                   exog = df[exog],
                   endog = None,
                   instruments = None).fit(cov_type='unadjusted')
print(res_first)

# correlation isn't huge, but F-statistic is pretty reasonable (F >> 10)

                            OLS Estimation Summary                            
Dep. Variable:       D8_10changeImmig   R-squared:                      0.0839
Estimator:                        OLS   Adj. R-squared:                 0.0838
No. Observations:               14901   F-statistic:                    1364.0
Date:                Wed, Apr 15 2020   P-value (F-stat)                0.0000
Time:                        19:40:35   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                                Parameter Estimates                                 
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
const               -0.4810     0.0074    -64.690     0.0000     -0.4956     -0.4664
D8_10changeCrime     0.3109 

In [40]:
# The second stage uses the instrument to fit the model.

dep = 'D8_10immigSelf'
exog = ['const']
endog = ['D8_10changeImmig']
instr = ['D8_10changeCrime']

res_second = IV2SLS(dependent = df[dep],
                    exog = df[exog],
                    endog = df[endog],
                    instruments = df[instr]).fit(cov_type='unadjusted')

print(res_second)
# DchangeImmig coefficienct is reduced to ~80% of previous

                          IV-2SLS Estimation Summary                          
Dep. Variable:         D8_10immigSelf   R-squared:                      0.0063
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0062
No. Observations:               14901   F-statistic:                    5.5622
Date:                Wed, Apr 15 2020   P-value (F-stat)                0.0184
Time:                        19:40:35   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                                Parameter Estimates                                 
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
const                0.0521     0.0300     1.7397     0.0819     -0.0066      0.1109
D8_10changeImmig    -0.1280 

In [41]:
# Sanity check with 'Direct' calculated from the difference between DchangeImmig and residuals from first stage

dep = 'D8_10immigSelf'
df['D8_10changeImmig_hat'] = df["D8_10changeImmig"] - res_first.resids
exog = ['const','D8_10changeImmig_hat']

res_direct = IV2SLS(dependent = df[dep],
                    exog = df[exog],
                    endog = None,
                    instruments = None).fit(cov_type='unadjusted')

print(compare({'OLS':res_ols, '2SLS':res_second, 'Direct': res_direct}))

# The 2SLS coefficient on D8_10changeImmig and the direct coefficient on D8_10changeImmig_hat are identical.

                                 Model Comparison                                
                                        OLS               2SLS             Direct
---------------------------------------------------------------------------------
Dep. Variable                D8_10immigSelf     D8_10immigSelf     D8_10immigSelf
Estimator                               OLS            IV-2SLS                OLS
No. Observations                      14901              14901              14901
Cov. Est.                        unadjusted         unadjusted         unadjusted
R-squared                            0.0065             0.0063             0.0004
Adj. R-squared                       0.0064             0.0062             0.0003
F-statistic                          97.465             5.5622             5.5292
P-value (F-stat)                     0.0000             0.0184             0.0187
======================     ================   ================   ================
const           

In [25]:
# If we had multiple exogenous variables you couldn't just rely on "big F in first stage" => strong first stage
# there is a helper function that calculation partial F (and a bunch of other partial measures) for the instrument
# upon your candidate endogenous value

print(res_second.first_stage)

# surprisingly memory intensive!

       First Stage Estimation Results      
                           D8_10changeImmig
-------------------------------------------
R-squared                            0.0839
Partial R-squared                    0.0839
Shea's R-squared                     0.0839
Partial F-statistic                  631.44
P-value (Partial F-stat)             0.0000
Partial F-stat Distn                chi2(1)
==========================      ===========
const                               -0.4810
                                  (-64.717)
D8_10changeCrime                     0.3109
                                   (25.129)
-------------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


In [29]:
# Can also specify the model with a formula

formula = ('D8_10immigSelf ~ 1 + [D8_10changeImmig ~ D8_10changeCrime]')
mod = IV2SLS.from_formula(formula, df)
res_formula = mod.fit(cov_type='unadjusted')

print(compare({'OLS':res_ols, '2SLS':res_second, 'Direct': res_direct, 'From Formula':res_formula}))

# Changes nothing other than that 'const' is renamed 'Intercept'

                                          Model Comparison                                          
                                        OLS               2SLS             Direct       From Formula
----------------------------------------------------------------------------------------------------
Dep. Variable                D8_10immigSelf     D8_10immigSelf     D8_10immigSelf     D8_10immigSelf
Estimator                               OLS            IV-2SLS                OLS            IV-2SLS
No. Observations                      14901              14901              14901              14901
Cov. Est.                        unadjusted         unadjusted         unadjusted         unadjusted
R-squared                            0.0065             0.0063             0.0004             0.0063
Adj. R-squared                       0.0064             0.0062             0.0003             0.0062
F-statistic                          97.465             5.5622             5.5292          

In [73]:
# It also handles categorical variables - adding controlImmigW9 as a categorical
# it drops the first category by default, so I'm setting the order so that the first category is the highest frequency
# "Some Control" is thus the base (shame it doesn't show that automatically - boo!)

df["controlImmigW9_cat"] = BES_Panel["controlImmigW9"]
df["controlImmigW9_cat"] = df["controlImmigW9_cat"].cat.reorder_categories( list(BES_Panel["controlImmigW9"].value_counts().index) )

dep = 'D8_10immigSelf'
exog = ['const','controlImmigW9_cat']
endog = ['D8_10changeImmig']
instr = ['D8_10changeCrime']

res_cat = IV2SLS(   dependent = df[dep],
                    exog = df[exog],
                    endog = df[endog],
                    instruments = df[instr]).fit(cov_type='unadjusted')

print(res_cat)

ValueError: regressors [exog endog] do not have full column rank

In [33]:
## Post-estimation diagnostics

In [42]:
# Is changeImmig actually endogeneous to immigSelf?

res_second.wooldridge_regression

## Nope

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 0.2732
P-value: 0.6012
Distributed: chi2(1)
WaldTestStatistic, id: 0xe9d53748

In [ ]:
# How much do I buy this as a measure of endogeneity? Quick sanity check - will test on "really obviously endogeneous" variables

In [44]:
# Add in DimmigEcon, DimmigCultural

df = pd.concat([D8_10immigSelf,D8_10changeImmig,D8_10changeCrime,D8_10immigEcon,D8_10immigCultural],axis=1).dropna()
df.columns = ["D8_10immigSelf","D8_10changeImmig","D8_10changeCrime","D8_10immigEcon","D8_10immigCultural"]
df = add_constant(df, has_constant='add')

In [46]:
dep = 'D8_10immigSelf'
exog = ['const']
endog = ['D8_10immigEcon']
instr = ['D8_10changeCrime',"D8_10changeImmig","D8_10immigCultural"] ## these are not necessarily effective/sensible instruments!

res_test = IV2SLS(  dependent = df[dep],
                    exog = df[exog],
                    endog = df[endog],
                    instruments = df[instr]).fit(cov_type='unadjusted')
print(res_test)

# F value looks bonkers, R-squared seems very low

                          IV-2SLS Estimation Summary                          
Dep. Variable:         D8_10immigSelf   R-squared:                     -0.0565
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0566
No. Observations:               14901   F-statistic:                    467.92
Date:                Wed, Apr 15 2020   P-value (F-stat)                0.0000
Time:                        19:44:29   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
const             -0.1775     0.0204    -8.6921     0.0000     -0.2175     -0.1375
D8_10immigEcon     0.6739     0.0312

In [47]:
# Is DimmigEcon endogeneous to DimmigSelf?

res_test.wooldridge_regression

# Yes, very much so

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 239.5133
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x71911f48

In [48]:
# How about those instruments? Too many or just enough?

res_test.wooldridge_overid

# Hella too many (/too correlated with the dependent variable)

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 14.4561
P-value: 0.0007
Distributed: chi2(2)
WaldTestStatistic, id: 0xe9d53548

In [49]:
# Lets drop DimmigCultural as an instrument

dep = 'D8_10immigSelf'
exog = ['const']
endog = ['D8_10immigEcon']
instr = ['D8_10changeCrime',"D8_10changeImmig"] 

res_test2 = IV2SLS(  dependent = df[dep],
                    exog = df[exog],
                    endog = df[endog],
                    instruments = df[instr]).fit(cov_type='unadjusted')
print(res_test2)

                          IV-2SLS Estimation Summary                          
Dep. Variable:         D8_10immigSelf   R-squared:                     -0.3980
Estimator:                    IV-2SLS   Adj. R-squared:                -0.3981
No. Observations:               14901   F-statistic:                    69.429
Date:                Wed, Apr 15 2020   P-value (F-stat)                0.0000
Time:                        19:49:38   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
const             -0.3992     0.0640    -6.2360     0.0000     -0.5247     -0.2738
D8_10immigEcon     1.1875     0.1425

In [51]:
res_test2.wooldridge_overid

# Bingo - now it's not overidentified
# Ironically, changeImmig is sufficiently exogeneous to immigSelf to be an instrument

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 0.0207
P-value: 0.8856
Distributed: chi2(1)
WaldTestStatistic, id: 0x71820088

In [52]:
# But it's not clear whether DchangeCrime/DchangeImmig fulfil the first criterion of 'instrumentality'

print(res_test2.first_stage)

      First Stage Estimation Results     
                           D8_10immigEcon
-----------------------------------------
R-squared                          0.0094
Partial R-squared                  0.0094
Shea's R-squared                   0.0094
Partial F-statistic                141.95
P-value (Partial F-stat)           0.0000
Partial F-stat Distn              chi2(2)
==========================    ===========
const                              0.3685
                                 (31.274)
D8_10changeCrime                   0.0049
                                 (0.3953)
D8_10changeImmig                  -0.1320
                                (-11.512)
-----------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


In [ ]:
# alternative endogeneity tests

In [59]:
res_test.durbin()
# Not robust to heteroscedasity (unlike wooldridge)

Durbin test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 239.0375
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0xeb1c3a08

In [60]:
res_test.wu_hausman()
# Slightly modified version of Durbin
# Also, not robust to heteroscedasity (unlike wooldridge)

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 242.8856
P-value: 0.0000
Distributed: F(1,14898)
WaldTestStatistic, id: 0xec89d888

In [61]:
# Robust to heteroscedasity
# but less power
res_test.wooldridge_score

Wooldridge's score test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 162.0395
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x717c9908

In [53]:
# Better covariance models? Yes - several are accessible - but the default one is "robust" (to heteroscedasticity)

dep = 'D8_10immigSelf'
exog = ['const']
endog = ['D8_10immigEcon']
instr = ['D8_10changeCrime',"D8_10changeImmig"] 

res_test3 = IV2SLS(  dependent = df[dep],
                    exog = df[exog],
                    endog = df[endog],
                    instruments = df[instr]).fit()
print(res_test3)

# Coefficients don't change, just the F-statistic/comparable error/T-stats get a bit more (accurately) conservative

                          IV-2SLS Estimation Summary                          
Dep. Variable:         D8_10immigSelf   R-squared:                     -0.3980
Estimator:                    IV-2SLS   Adj. R-squared:                -0.3981
No. Observations:               14901   F-statistic:                    54.874
Date:                Wed, Apr 15 2020   P-value (F-stat)                0.0000
Time:                        19:59:38   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
const             -0.3992     0.0703    -5.6775     0.0000     -0.5371     -0.2614
D8_10immigEcon     1.1875     0.1603

In [ ]:
# Conclusion - instrumenting doesn't seem necessary for DchangeImmig => DimmigSelf

In [7]:
df

,const,D8_10immigSelf,D8_10changeImmig,D8_10changeCrime,D8_10immigEcon,D8_10immigCultural
1,1.0,2.0,1.0,2.0,0.0,0.0
3,1.0,-1.0,0.0,0.0,0.0,0.0
6,1.0,-1.0,-1.0,0.0,0.0,0.0
8,1.0,0.0,0.0,-1.0,0.0,0.0
9,1.0,0.0,-1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
97124,1.0,-2.0,0.0,0.0,-3.0,-2.0
97128,1.0,0.0,-1.0,0.0,0.0,0.0
97142,1.0,0.0,0.0,0.0,0.0,0.0
97146,1.0,0.0,0.0,0.0,-1.0,2.0


In [6]:
# Better covariance models? Yes - several are accessible - but the default one is "robust" (to heteroscedasticity)

dep = 'D8_10immigSelf'
exog = ['const',]
endog = ['D8_10immigEcon',]
instr = ['D8_10changeCrime',"D8_10changeImmig"] 

res_test3 = IV2SLS( dependent = df[dep],
                    exog = df[exog],
                    endog = df[endog],
                    instruments = df[instr]).fit()
print(res_test3)

# Coefficients don't change, just the F-statistic/comparable error/T-stats get a bit more (accurately) conservative

                          IV-2SLS Estimation Summary                          
Dep. Variable:         D8_10immigSelf   R-squared:                     -0.3980
Estimator:                    IV-2SLS   Adj. R-squared:                -0.3981
No. Observations:               14901   F-statistic:                    54.874
Date:                Wed, May 06 2020   P-value (F-stat)                0.0000
Time:                        16:39:50   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
const             -0.3992     0.0703    -5.6775     0.0000     -0.5371     -0.2614
D8_10immigEcon     1.1875     0.1603

In [ ]:
#############################################

In [37]:
df.columns

Index(['const', 'D8_10immigSelf', 'D8_10changeImmig', 'D8_10changeCrime',
       'D8_10immigEcon', 'D8_10immigCultural', 'D9_10controlImmig',
       'immigSelfW8', 'immigSelfW10', 'changeImmigW8', 'changeImmigW10',
       'changeCrimeW8', 'changeCrimeW10', 'immigEconW8', 'immigEconW10',
       'immigCulturalW8', 'immigCulturalW10', 'controlImmigW8',
       'controlImmigW9', 'controlImmigW10', 'changeImmigW8_cat',
       'changeImmigW10_cat', 'controlImmigW8_cat', 'controlImmigW9_cat',
       'controlImmigW10_cat'],
      dtype='object')

In [94]:
# Better covariance models? Yes - several are accessible - but the default one is "robust" (to heteroscedasticity)

dep = 'D8_10immigSelf'
exog = ['const']
endog = ["D8_10changeImmig",]
instr = ['D8_10changeCrime',] 


mask = df[[dep]+exog+endog+instr].notnull().all(axis=1)
# make sure we're only ever looking at rows with no nans

res =      IV2SLS(  dependent = df[dep][mask],
                    exog = df[exog][mask],
                    endog = df[endog][mask],
                    instruments = df[instr][mask]).fit()
print(res)

# Coefficients don't change, just the F-statistic/comparable error/T-stats get a bit more (accurately) conservative

                          IV-2SLS Estimation Summary                          
Dep. Variable:         D8_10immigSelf   R-squared:                      0.0057
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0056
No. Observations:               12140   F-statistic:                    2.7853
Date:                Wed, May 06 2020   P-value (F-stat)                0.0951
Time:                        19:35:23   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                                Parameter Estimates                                 
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
const                0.0561     0.0377     1.4877     0.1368     -0.0178      0.1300
D8_10changeImmig    -0.1173 

In [96]:
# Better covariance models? Yes - several are accessible - but the default one is "robust" (to heteroscedasticity)

dep = 'D8_10immigEcon'
exog = ['const']
endog = ["D8_10changeImmig",]
instr = ['D8_10changeCrime',] 


mask = df[[dep]+exog+endog+instr].notnull().all(axis=1)
# make sure we're only ever looking at rows with no nans

res =      IV2SLS(  dependent = df[dep][mask],
                    exog = df[exog][mask],
                    endog = df[endog][mask],
                    instruments = df[instr][mask]).fit()
print(res)

# Coefficients don't change, just the F-statistic/comparable error/T-stats get a bit more (accurately) conservative

                          IV-2SLS Estimation Summary                          
Dep. Variable:         D8_10immigEcon   R-squared:                      0.0073
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0073
No. Observations:               12140   F-statistic:                    3.7252
Date:                Wed, May 06 2020   P-value (F-stat)                0.0536
Time:                        19:36:09   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                                Parameter Estimates                                 
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
const                0.3886     0.0257     15.134     0.0000      0.3383      0.4390
D8_10changeImmig    -0.0929 

In [97]:
# Better covariance models? Yes - several are accessible - but the default one is "robust" (to heteroscedasticity)

dep = 'D8_10immigCultural'
exog = ['const']
endog = ["D8_10changeImmig",]
instr = ['D8_10changeCrime',] 


mask = df[[dep]+exog+endog+instr].notnull().all(axis=1)
# make sure we're only ever looking at rows with no nans

res =      IV2SLS(  dependent = df[dep][mask],
                    exog = df[exog][mask],
                    endog = df[endog][mask],
                    instruments = df[instr][mask]).fit()
print(res)

# Coefficients don't change, just the F-statistic/comparable error/T-stats get a bit more (accurately) conservative

                          IV-2SLS Estimation Summary                          
Dep. Variable:     D8_10immigCultural   R-squared:                      0.0012
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0011
No. Observations:               12140   F-statistic:                    0.0754
Date:                Wed, May 06 2020   P-value (F-stat)                0.7837
Time:                        19:36:46   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                                Parameter Estimates                                 
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
const                0.2179     0.0259     8.4079     0.0000      0.1671      0.2687
D8_10changeImmig    -0.0131 

In [47]:
BES_Panel["immigSelfW9"].notnull().sum()

30036

In [ ]:
## Going to look at cross-lagged correlation
# picking up on the bleeding edge of causal analysis ... circa 1970
# (everything after that is article saying why this a rubbish approach)
# but it seems weird to skip something so very simple!

In [3]:
immigSelfW10 = search(BES_reduced_with_na,"immigSelfW10").index[0]
immigSelfW8  = search(BES_reduced_with_na,"immigSelfW8" ).index[0]
D8_10immigSelf = BES_reduced_with_na[immigSelfW10]-BES_reduced_with_na[immigSelfW8]

changeImmigW8  = search(BES_reduced_with_na,"changeImmigW8" ).index[0]
changeImmigW10 = search(BES_reduced_with_na,"changeImmigW10").index[0]
D8_10changeImmig = BES_reduced_with_na[changeImmigW10]-BES_reduced_with_na[changeImmigW8]

changeCrimeW8  = search(BES_reduced_with_na,"changeCrimeW8" ).index[0]
changeCrimeW10 = search(BES_reduced_with_na,"changeCrimeW10").index[0]
D8_10changeCrime = BES_reduced_with_na[changeCrimeW10]-BES_reduced_with_na[changeCrimeW8]

immigEconW10 = search(BES_reduced_with_na,"immigEconW10").index[0]
immigEconW8  = search(BES_reduced_with_na,"immigEconW8" ).index[0]
D8_10immigEcon = BES_reduced_with_na[immigEconW10]-BES_reduced_with_na[immigEconW8]

immigCulturalW10 = search(BES_reduced_with_na,"immigCulturalW10").index[0]
immigCulturalW8  = search(BES_reduced_with_na,"immigCulturalW8" ).index[0]
D8_10immigCultural = BES_reduced_with_na[immigCulturalW10]-BES_reduced_with_na[immigCulturalW8]


# df = pd.concat([D8_10immigSelf,D8_10changeImmig,D8_10changeCrime,D8_10immigEcon,D8_10immigCultural],axis=1).dropna()
# df.columns = ["D8_10immigSelf","D8_10changeImmig","D8_10changeCrime","D8_10immigEcon","D8_10immigCultural"]

In [4]:
BES_reduced_with_na[immigSelfW10].astype('float32').corr(BES_reduced_with_na[immigEconW8].astype('float32')),\
    BES_reduced_with_na[immigSelfW8].astype('float32').corr(BES_reduced_with_na[immigEconW10].astype('float32')),\


(0.7129805827519166, 0.6614250724714742)

In [5]:
immigSelfW10_ser = BES_reduced_with_na[immigSelfW10].astype('float32')
immigSelfW8_ser  = BES_reduced_with_na[immigSelfW8].astype('float32')
immigEconW8_ser  = BES_reduced_with_na[immigEconW8].astype('float32')
immigEconW10_ser = BES_reduced_with_na[immigEconW10].astype('float32')
mask = immigSelfW10_ser.notnull()&immigSelfW8_ser.notnull()&immigEconW8_ser.notnull()&immigEconW10_ser.notnull()

immigSelfW10_ser[mask].corr(immigEconW8_ser[mask]),\
    immigSelfW8_ser[mask].corr(immigEconW10_ser[mask])

# marginal implication of immigEcon -> immigSelf


(0.7188305330703965, 0.672131947195341)

In [6]:
var1 = "immigEcon"
var2 = "immigSelf"


def cross_lagged_corr(var1,var2,wave1="W8",wave2="W10"):
    immigEconW10 = search(BES_reduced_with_na,var1+wave2).index[0]
    immigEconW8  = search(BES_reduced_with_na,var1+wave1).index[0]
    immigSelfW10 = search(BES_reduced_with_na,var2+wave2).index[0]
    immigSelfW8  = search(BES_reduced_with_na,var2+wave1).index[0]


    immigSelfW10_ser = BES_reduced_with_na[immigSelfW10].astype('float32')
    immigSelfW8_ser  = BES_reduced_with_na[immigSelfW8].astype('float32')
    immigEconW8_ser  = BES_reduced_with_na[immigEconW8].astype('float32')
    immigEconW10_ser = BES_reduced_with_na[immigEconW10].astype('float32')
    mask = immigSelfW10_ser.notnull()&immigSelfW8_ser.notnull()&immigEconW8_ser.notnull()&immigEconW10_ser.notnull()

    var1_var2_corr = immigSelfW10_ser[mask].corr(immigEconW8_ser[mask])
    var2_var1_corr = immigEconW10_ser[mask].corr(immigSelfW8_ser[mask])

    print(var1+wave1+"->"+var2+wave2,var1_var2_corr)
    print(var2+wave1+"->"+var1+wave2,var2_var1_corr)

# marginal implication of immigEcon -> immigSelf
cross_lagged_corr(var1,var2)

immigEconW8->immigSelfW10 0.7188305330703965
immigSelfW8->immigEconW10 0.672131947195341


In [17]:
var1 = "immigCultural"
var2 = "immigSelf"

cross_lagged_corr(var1,var2)

immigCulturalW8->immigSelfW10 0.7292689635783719
immigSelfW8->immigCulturalW10 0.7068053396270592


In [16]:
var1 = "immigCultural"
var2 = "immigEcon"

cross_lagged_corr(var1,var2,"W1","W2")

immigCulturalW1->immigEconW2 0.7393625171073076
immigEconW1->immigCulturalW2 0.7244905616271176


In [18]:
var1 = "immigCultural"
var2 = "immigEcon"

cross_lagged_corr(var1,var2,"W7","W8")

immigCulturalW7->immigEconW8 0.7440887063542656
immigEconW7->immigCulturalW8 0.745748299358296


In [19]:
var1 = "immigCultural"
var2 = "immigEcon"

cross_lagged_corr(var1,var2,"W16","W17")

immigCulturalW16->immigEconW17 0.7172137674311125
immigEconW16->immigCulturalW17 0.7280155092150624


In [20]:
var1 = "immigCultural"
var2 = "immigEcon"

cross_lagged_corr(var1,var2,"W1","W10")

immigCulturalW1->immigEconW10 0.6403261561551364
immigEconW1->immigCulturalW10 0.6678046745596167


In [9]:
var1 = "immigCultural"
var2 = "immigEcon"

cross_lagged_corr(var1,var2,"W1","W17")

immigCulturalW1->immigEconW17 0.6092705540273134
immigEconW1->immigCulturalW17 0.625516171687894


In [7]:
var1 = "immigSelf"
var2 = "immigEcon"

cross_lagged_corr(var1,var2,"W7","W17")

immigSelfW7->immigEconW17 0.6203347963846528
immigEconW7->immigSelfW17 0.708097746600024


In [8]:
var1 = "immigSelf"
var2 = "immigCultural"

cross_lagged_corr(var1,var2,"W7","W17")

immigSelfW7->immigCulturalW17 0.6690048315463473
immigCulturalW7->immigSelfW17 0.6982389225885333


In [22]:
var1 = "immigSelf"
var2 = "immigEcon"

cross_lagged_corr(var1,var2,"W16","W17")

immigSelfW16->immigEconW17 0.6959143303053202
immigEconW16->immigSelfW17 0.7189111196006936


In [10]:
var1 = "immigSelf"
var2 = "immigCultural"

cross_lagged_corr(var1,var2,"W8","W10")

immigSelfW8->immigCulturalW10 0.7068053396270592
immigCulturalW8->immigSelfW10 0.7292689635783719


In [25]:
var1 = "immigrantsWelfareState"
var2 = "immigCultural"

cross_lagged_corr(var1,var2,"W8","W10")

immigrantsWelfareStateW8->immigCulturalW10 -0.6691791319586687
immigCulturalW8->immigrantsWelfareStateW10 -0.6609960512590991


In [28]:
var1 = "ukCoopMovement"
var2 = "immigSelf"

cross_lagged_corr(var1,var2,"W8","W10")

ukCoopMovementW8->immigSelfW10 -0.6591330822483417
immigSelfW8->ukCoopMovementW10 -0.622298746153174


In [29]:
var1 = "changeImmig"
var2 = "immigrantsWelfareState"

cross_lagged_corr(var1,var2,"W8","W10")

changeImmigW8->immigrantsWelfareStateW10 0.49436309713279547
immigrantsWelfareStateW8->changeImmigW10 0.3864699872486674


In [30]:
var1 = "controlImmig"
var2 = "immigEcon"

cross_lagged_corr(var1,var2,"W8","W10")

controlImmigW8->immigEconW10 0.4982253242822512
immigEconW8->controlImmigW10 -0.34692775986003993


In [38]:
var1 = "controlImmig"
var2 = "immigSelf"

cross_lagged_corr(var1,var2,"W8","W9")

controlImmigW8->immigSelfW9 0.5606340785481458
immigSelfW8->controlImmigW9 -0.4565388083113115


In [37]:
var1 = "controlImmig"
var2 = "immigSelf"

cross_lagged_corr(var1,var2,"W9","W17")

controlImmigW9->immigSelfW17 -0.45100966137563586
immigSelfW9->controlImmigW17 -0.39225566163412734


In [39]:
var1 = "immigContributeTake"
var2 = "immigSelf"

cross_lagged_corr(var1,var2,"W7","W8")

immigContributeTakeW7->immigSelfW8 -0.6359327433745416
immigSelfW7->immigContributeTakeW8 -0.6248992810536302


In [40]:
var1 = "immigContributeTake"
var2 = "immigEcon"

cross_lagged_corr(var1,var2,"W7","W8")

immigContributeTakeW7->immigEconW8 -0.6410224260083954
immigEconW7->immigContributeTakeW8 -0.641945318138139


In [41]:
var1 = "immigContributeTake"
var2 = "immigCultural"

cross_lagged_corr(var1,var2,"W7","W8")

immigContributeTakeW7->immigCulturalW8 -0.5960650555769174
immigCulturalW7->immigContributeTakeW8 -0.5998208576139431


In [42]:
var1 = "immigContributeTake"
var2 = "immigrantsWelfareState"

cross_lagged_corr(var1,var2,"W7","W8")

immigContributeTakeW7->immigrantsWelfareStateW8 0.6907696790871389
immigrantsWelfareStateW7->immigContributeTakeW8 0.6962436959138493


In [46]:
var1 = "europeanness"
var2 = "immigSelf"

cross_lagged_corr(var1,var2,"W8","W10")

europeannessW8->immigSelfW10 0.5518217874645736
immigSelfW8->europeannessW10 0.5509476669793407


In [ ]:
## how to find instruments automatically?
# look for things with high_corr to your endog variable and low_corr with your target variable

In [23]:
search(BES_reduced_with_na,"controlImmigW8")

controlImmigW8__Complete control    8014
dtype: int64

In [26]:
%%time
# BES_reduced_with_na["trust_per_resp"] = trust_per_resp
pattern = "controlImmigW8__Complete control"
name = "controlImmigW8__Complete control"
corr_type = "spearman"
corr_df,corr_type = make_corr_summary(BES_reduced_with_na, name, corr_type, pattern=pattern,min_sample_size=500)
display_corr(corr_df, name, corr_type, text_wrap_length=50)

,r,p,N
immigSelfW8__Allow many more,0.610000,0.00,7460
immigEconW8__Good for economy,0.570000,0.00,7727
approveEUW9__Strongly approve,0.560000,0.00,6419
immigCulturalW8__Enriches cultural life,0.560000,0.00,7809
euUKRichW8__Strongly agree,0.560000,0.00,7478
euLawsLevelW8__Strongly agree,0.530000,0.00,7667
euMoreW8__Many more,0.530000,0.00,7650
dealVremainW18_Remain in the EU,0.510000,0.00,3277
remainVnodealW16_Remain in the EU,0.510000,0.00,2595
euPreventWarW8__Strongly agree,0.510000,0.00,7503


Wall time: 3min 9s


In [34]:
controlImmig_corrs = corr_df.copy()

In [35]:
%%time
# BES_reduced_with_na["trust_per_resp"] = trust_per_resp
pattern = "immigSelfW8__Allow many more"
name = "immigSelfW8__Allow many more"
corr_type = "spearman"
corr_df,corr_type = make_corr_summary(BES_reduced_with_na, name, corr_type, pattern=pattern,min_sample_size=500)
display_corr(corr_df, name, corr_type, text_wrap_length=50)

,r,p,N
immigSelfW7__Allow many more,0.820000,0.00,20527
asylumMoreW8__Many more,0.780000,0.00,29436
immigEconW8__Good for economy,0.750000,0.00,29449
immigCulturalW8__Enriches cultural life,0.750000,0.00,29690
benInteg3W11__I fully support it.,0.750000,0.00,943
euMoreW8__Many more,0.720000,0.00,29581
familiesMoreW8__Many more,0.710000,0.00,29297
benInteg5W11__I fully support it.,0.690000,0.00,894
benInteg1W11__I fully support it.,0.680000,0.00,948
immigrationLevelW4__Increased a lot,0.670000,0.00,17210


Wall time: 8min 46s


In [48]:
immigSelf_corrs = corr_df.copy()

In [49]:
immigSelf_corrs.shape

(2617, 4)

In [50]:
controlImmig_corrs.shape

(1577, 4)

In [51]:
overlap_corrs = set(immigSelf_corrs.index).intersection(set(controlImmig_corrs.index))

In [56]:
(controlImmig_corrs.loc[overlap_corrs] - immigSelf_corrs.loc[overlap_corrs]).sort_values(by='abs_r').tail(50)

,r,p,N,abs_r
possibleTradeNIW15_Possible,-0.02,0.00,-2875,0.02
noDealResults_3W15_Yes,-0.02,0.00,-3035,0.02
cabinetSupportW7__Mainly leave,0.02,0.00,-13717,0.02
generalElectionVotePostW18_Liberal Democrat,0.02,0.00,-1621,0.02
expectGoodConductEURefW7__Conducted unfairly,-0.02,0.00,-14776,0.02
deliversResultW16__Completely,0.02,0.00,-6506,0.02
campaigninfoRemainW8__Strongly agree,0.03,0.00,-21279,0.03
euDominanceW8__Many more powers,-0.03,0.00,-9670,0.03
remainToneW8__5 - Hope,0.03,0.00,-20963,0.03
JACS_subjectW19_V,0.03,0.00,-4523,0.03


In [109]:
[x for x in df.columns if 'cat' in x]

['changeImmigW8_cat',
 'changeImmigW10_cat',
 'controlImmigW8_cat',
 'controlImmigW9_cat',
 'controlImmigW10_cat']

In [99]:
[x for x in df.columns if x[0]!="D"]

['const',
 'immigSelfW8',
 'immigSelfW9',
 'immigSelfW10',
 'changeImmigW8',
 'changeImmigW10',
 'changeCrimeW8',
 'changeCrimeW10',
 'immigEconW8',
 'immigEconW10',
 'immigCulturalW8',
 'immigCulturalW10',
 'controlImmigW8',
 'controlImmigW9',
 'controlImmigW10',
 'changeImmigW8_cat',
 'changeImmigW10_cat',
 'controlImmigW8_cat',
 'controlImmigW9_cat',
 'controlImmigW10_cat']

In [105]:
small_test_set_no_deltas = ['immigSelfW8', 'immigSelfW9', 'immigSelfW10', 'changeImmigW8', 'changeImmigW10', 'changeCrimeW8', 'changeCrimeW10', 'immigEconW8', 'immigEconW10', 'immigCulturalW8', 'immigCulturalW10', 'controlImmigW8', 'controlImmigW9', 'controlImmigW10']
df[small_test_set_no_deltas].to_csv(BES_data_folder+"Portes"+os.sep+"small_test_set_no_deltas.csv",index_label = False)

In [116]:
df[small_test_set_no_deltas].dropna().to_csv(BES_data_folder+"Portes"+os.sep+"small_test_set_no_deltas_no_nans.csv",index_label = False)

In [107]:
df[small_test_set_no_deltas].dropna()[0:100].to_csv(BES_data_folder+"Portes"+os.sep+"small_test_set_no_deltas_no_nans_few_rows.csv",index_label = False)

In [117]:
len([x for x in df.columns if 'cat' in x])

5

In [113]:
mixed_variable_types = small_test_set_no_deltas + [x for x in df.columns if 'cat' in x]

In [114]:
mixed_variable_types

['immigSelfW8',
 'immigSelfW9',
 'immigSelfW10',
 'changeImmigW8',
 'changeImmigW10',
 'changeCrimeW8',
 'changeCrimeW10',
 'immigEconW8',
 'immigEconW10',
 'immigCulturalW8',
 'immigCulturalW10',
 'controlImmigW8',
 'controlImmigW9',
 'controlImmigW10',
 'changeImmigW8_cat',
 'changeImmigW10_cat',
 'controlImmigW8_cat',
 'controlImmigW9_cat',
 'controlImmigW10_cat']

In [115]:
df[mixed_variable_types].dropna()[0:100].to_csv(BES_data_folder+"Portes"+os.sep+"mixed_variable_types_no_nans_few_rows.csv",index_label = False)

In [119]:
df[small_test_set_no_deltas].dropna()[0:100]

immigSelfW8         Int8
immigSelfW9         Int8
immigSelfW10        Int8
changeImmigW8       Int8
changeImmigW10      Int8
changeCrimeW8       Int8
changeCrimeW10      Int8
immigEconW8         Int8
immigEconW10        Int8
immigCulturalW8     Int8
immigCulturalW10    Int8
controlImmigW8      Int8
controlImmigW9      Int8
controlImmigW10     Int8
dtype: object

In [125]:
df[df[small_test_set_no_deltas].columns[0:5]] = (df[df[small_test_set_no_deltas].columns[0:5]]/10).astype('float32')
df[small_test_set_no_deltas].dropna()[0:100].to_csv(BES_data_folder+"Portes"+os.sep+"float_types_added_no_nans_few_rows.csv",index_label = False)

In [129]:
df[small_test_set_no_deltas].dropna().dtypes.value_counts()

Int8       9
float32    5
dtype: int64